# Hybrid Zonotopes

In [1]:
# Generic TLT imports
from pyspect import *
from pyspect.langs.ltl import *
# Hybrid Zonotope imports
from hz_reachability.hz_impl import HZImpl
from hz_reachability.systems.cars import CarLinearModel2D
from hz_reachability.shapes import HZShapes
from hz_reachability.spaces import ParkingSpace

TLT.select(ContinuousLTL)

## Environment definition

In [2]:
# Option 1: Use the existing set templates or cretate your own (Not implemented for HZ yet).
# e.g., state_space = ReferredSet('state_space')

# Option 2: Use the generic Set method to import any custom shape
shapes = HZShapes()
center = Set(shapes.center())
road_west = Set(shapes.road_west())
road_east = Set(shapes.road_east())
road_north = Set(shapes.road_north())
road_south = Set(shapes.road_south())

## Definitions

### Task

In [3]:
# Example task: Stay in road_e, or road_n UNTIL you REACH exit_n.
task = Until(Or(road_east, road_north), center)

### Dynamics

In [4]:
reach_dynamics = CarLinearModel2D()

### Implementation

In [5]:
# Hybrid Zonotope implementation
impl = HZImpl(dynamics=reach_dynamics, space = ParkingSpace(), time_horizon = 5)

## Solve

- `construct(task)`: Take an LTL, a set, or a lazy set, or an already constructed TLT and make sure it is a valid TLT object. Basically construct the compute graph for the given task.
- `realize(impl)`: initiates the actual computations.
- `out`: The final set in your specific set implementation. e.g., it would be a hybrid zonotope.

In [6]:
# Solve the problem - Find the states that can satisfy the task
out = TLT.construct(task).realize(impl)

print(f'out = {out}')
print(f'Gc = \n{out.Gc}')

out = <hz_reachability.sets.HybridZonotope object at 0x7f0dd892ed80>
Gc = 
[[0.05 0.   0.   ... 0.   0.   0.  ]
 [0.   0.95 0.   ... 0.   0.   0.  ]]


## Conversion

### HJ Setup

In [9]:
import hj_reachability as hj

from pyspect.impls.hj_reachability import HJImpl
from hj_reachability.systems import Bicycle4D
from pyspect.plotting.levelsets import *

from math import pi

# Define origin and size of area, makes it easier to scale up/down later on 
X0, XN = -1.2, 2.4
Y0, YN = -1.2, 2.4

min_bounds = np.array([   X0,    Y0, +0.3, +0.3])
max_bounds = np.array([XN+X0, YN+Y0, +0.8, +0.8])
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(min_bounds, max_bounds),
                                                               (31, 31, 15, 15),
                                                               periodic_dims=2)

dynamics = dict(cls=Bicycle4D,
                min_steer=-pi/4, 
                max_steer=+pi/4,
                min_accel=-0.4,
                max_accel=+0.4)

impl = HJImpl(dynamics, grid, 3)
impl.set_axes_names('t', 'x', 'y', 'vx', 'vy')

### Method

In [ ]:
import hj_reachability.shapes as shp

vf = shp.point(grid, out.C)